# Weaviate Vector Database Connection Test

This notebook demonstrates **step-by-step** how to check if your Weaviate vector database connection is successful.

## What This Notebook Does:
✅ Connects to Weaviate running on localhost (Docker)  
✅ Verifies connection status  
✅ Checks database readiness  
✅ Lists all collections  
✅ Tests basic queries  
✅ Properly closes the connection  

## Prerequisites:
- Weaviate running in Docker: `docker-compose up -d`
- Required Python packages installed: `weaviate-client`, `python-dotenv`

## Step 1: Import Required Libraries

Import all necessary libraries for connecting to Weaviate vector database.

In [ ]:
# Import weaviate client library for vector database operations
import weaviate

# Import os for environment variable access
import os

# Import dotenv to load environment variables from .env file
from dotenv import load_dotenv

# Import requests to check REST API endpoint
import requests

# Import json for pretty printing responses
import json

# Print confirmation
print("✅ All libraries imported successfully!")
print(f"📦 Weaviate client version: {weaviate.__version__}")

## Step 2: Load Environment Variables (Optional)

Load configuration from .env file if you have one. This is optional for local development.

In [ ]:
# Load environment variables from .env file (if exists)
load_dotenv()

# Define connection parameters
# These are the default values for Weaviate running in Docker locally
WEAVIATE_HOST = os.getenv("WEAVIATE_HOST")
WEAVIATE_PORT = int(os.getenv("WEAVIATE_PORT"))
WEAVIATE_GRPC_PORT = int(os.getenv("WEAVIATE_GRPC_PORT"))

print("📝 Connection Configuration:")
print(f"   Host: {WEAVIATE_HOST}")
print(f"   REST Port: {WEAVIATE_PORT}")
print(f"   gRPC Port: {WEAVIATE_GRPC_PORT}")
print(f"   REST URL: http://{WEAVIATE_HOST}:{WEAVIATE_PORT}")
print("✅ Configuration loaded successfully!")

## Step 3: Check if Weaviate Docker Container is Running

Before connecting, verify that Weaviate is actually running by checking the REST API endpoint.

In [ ]:
# Check if Weaviate is accessible via REST API
try:
    # Send GET request to Weaviate health check endpoint
    response = requests.get(f"http://{WEAVIATE_HOST}:{WEAVIATE_PORT}/v1/.well-known/ready", timeout=5)
    
    # Check if response is successful
    if response.status_code == 200:
        print("✅ SUCCESS: Weaviate is running and accessible!")
        print(f"   Status Code: {response.status_code}")
        print(f"   Response: {response.text}")
    else:
        print(f"⚠️ WARNING: Weaviate responded but with status code: {response.status_code}")
        
except requests.exceptions.ConnectionError:
    print("❌ ERROR: Cannot connect to Weaviate!")
    print(f"   Weaviate is NOT running at http://{WEAVIATE_HOST}:{WEAVIATE_PORT}")
    print("\n💡 Solution:")
    print("   1. Start Weaviate Docker container: docker-compose up -d")
    print("   2. Wait 30 seconds for initialization")
    print("   3. Run this cell again")
    
except Exception as e:
    print(f"❌ ERROR: {type(e).__name__}: {e}")

## Step 4: Connect to Weaviate Vector Database

Establish a connection to the Weaviate vector database using the Python client.

In [ ]:
# Initialize Weaviate client
try:
    # Connect to local Weaviate instance
    # This uses both REST API (port 8080) and gRPC (port 50051)
    client = weaviate.connect_to_local(
        host=WEAVIATE_HOST,      # Docker container host
        port=WEAVIATE_PORT,      # REST API port
        grpc_port=WEAVIATE_GRPC_PORT,  # gRPC port for v4 client
    )
    
    print("✅ SUCCESS: Weaviate client created!")
    print(f"   Client Type: {type(client)}")
    print(f"   Connection Method: weaviate.connect_to_local()")
    
except Exception as e:
    print(f"❌ ERROR: Failed to create Weaviate client")
    print(f"   Error Type: {type(e).__name__}")
    print(f"   Error Message: {e}")
    print("\n💡 Common Issues:")
    print("   - Weaviate not running: docker-compose up -d")
    print("   - Port 8080 or 50051 blocked by firewall")
    print("   - Wrong host/port configuration")
    client = None

## Step 5: Verify Database is Ready

Check if the Weaviate database is ready to accept queries.

In [ ]:
# Check if client is connected and database is ready
if client is not None:
    try:
        # Use is_ready() method to verify database status
        is_ready = client.is_ready()
        
        if is_ready:
            print("✅ SUCCESS: Weaviate database is READY!")
            print(f"   Ready Status: {is_ready}")
            print(f"   Database is fully operational and can accept queries")
        else:
            print("⚠️ WARNING: Weaviate client connected but database NOT ready")
            print("   Database might still be initializing...")
            
    except Exception as e:
        print(f"❌ ERROR: Failed to check database readiness")
        print(f"   Error: {e}")
else:
    print("❌ ERROR: Client is None - connection was not established")
    print("   Please run the previous cell to create the client first")

## Step 6: List All Available Collections

Retrieve and display all collections (schemas) in the vector database.

In [ ]:
# List all collections in the vector database
if client is not None and client.is_ready():
    try:
        # Get list of all collection names
        collections = client.collections.list_all()
        
        print(f"📊 Total Collections Found: {len(collections)}")
        print("=" * 50)
        
        if collections:
            print("\n✅ Available Collections:")
            for i, collection_name in enumerate(collections, 1):
                print(f"   {i}. {collection_name}")
                
                # Try to get count for each collection
                try:
                    collection = client.collections.get(collection_name)
                    result = collection.aggregate.over_all(total_count=True)
                    count = result.total_count
                    print(f"      └─ Document Count: {count}")
                except Exception as e:
                    print(f"      └─ Could not get count: {e}")
        else:
            print("\nℹ️ No collections found in the database")
            print("   This is normal for a fresh Weaviate installation")
            print("   Collections will appear after you create them")
            
    except Exception as e:
        print(f"❌ ERROR: Failed to list collections")
        print(f"   Error: {e}")
else:
    print("❌ ERROR: Client not ready")
    print("   Make sure previous cells executed successfully")

## Step 7: Test Basic Query Operations

Perform a test query to verify the database connection is fully functional.

In [ ]:
# Test if we can perform queries on the database
if client is not None and client.is_ready():
    try:
        # Get all collections again
        collections = client.collections.list_all()
        
        if collections:
            # Test with the first collection found
            test_collection_name = collections[0]
            print(f"🧪 Testing query operations with collection: '{test_collection_name}'")
            print("=" * 50)
            
            # Get collection reference
            collection = client.collections.get(test_collection_name)
            
            # Perform aggregation to count documents
            result = collection.aggregate.over_all(total_count=True)
            total_count = result.total_count
            
            print(f"\n✅ Query Test SUCCESSFUL!")
            print(f"   Collection: {test_collection_name}")
            print(f"   Total Documents: {total_count}")
            print(f"   Query Type: Aggregation (count)")
            print(f"\n🎉 Database connection is FULLY FUNCTIONAL!")
            
        else:
            print("ℹ️ No collections available to test queries")
            print("   Database connection is working, but no data to query yet")
            print("\n💡 To test with data:")
            print("   1. Create a collection using the API")
            print("   2. Add some documents")
            print("   3. Run this cell again")
            
    except Exception as e:
        print(f"❌ ERROR: Query test failed")
        print(f"   Error: {e}")
        print("\n💡 This might indicate:")
        print("   - Collection access issues")
        print("   - Permission problems")
        print("   - Database not fully initialized")
else:
    print("❌ ERROR: Cannot perform query test")
    print("   Client is not ready or not connected")

## Step 8: Connection Summary

Display a comprehensive summary of the connection test results.

In [ ]:
# Display comprehensive connection summary
print("=" * 60)
print("🔍 WEAVIATE CONNECTION TEST SUMMARY")
print("=" * 60)

# Check 1: Client Status
if client is not None:
    print("✅ Client Created: YES")
else:
    print("❌ Client Created: NO")

# Check 2: Database Ready
try:
    if client is not None:
        ready_status = client.is_ready()
        if ready_status:
            print("✅ Database Ready: YES")
        else:
            print("⚠️ Database Ready: NO (still initializing)")
    else:
        print("❌ Database Ready: Cannot check (no client)")
except:
    print("❌ Database Ready: Cannot check (error)")

# Check 3: Collections Available
try:
    if client is not None and client.is_ready():
        collections = client.collections.list_all()
        print(f"📊 Collections Found: {len(collections)}")
        if collections:
            for col in collections:
                print(f"   - {col}")
    else:
        print("📊 Collections Found: Cannot check")
except:
    print("📊 Collections Found: Error retrieving")

# Check 4: Configuration
print(f"\n⚙️ Configuration:")
print(f"   Host: {WEAVIATE_HOST}")
print(f"   REST Port: {WEAVIATE_PORT}")
print(f"   gRPC Port: {WEAVIATE_GRPC_PORT}")

# Final Status
print("\n" + "=" * 60)
if client is not None and client.is_ready():
    print("🎉 RESULT: CONNECTION SUCCESSFUL!")
    print("   Your Weaviate vector database is ready to use.")
else:
    print("⚠️ RESULT: CONNECTION ISSUES DETECTED")
    print("   Please review the errors above and fix them.")
print("=" * 60)

## Step 9: Close Connection Properly

Always close the Weaviate client connection when done to free up resources.

In [ ]:
# Close the Weaviate client connection gracefully
if client is not None:
    try:
        # Close the connection
        client.close()
        print("✅ SUCCESS: Weaviate connection closed successfully")
        print("   Resources have been freed")
        print("   Client is no longer usable")
        
    except Exception as e:
        print(f"⚠️ WARNING: Error while closing connection")
        print(f"   Error: {e}")
        print("   Connection might already be closed or invalid")
else:
    print("ℹ️ INFO: No client to close (client was None)")

---

## 📚 Next Steps

### If Connection Successful ✅:
1. **Create Collections** - Define your data schema
2. **Add Documents** - Insert vector embeddings and metadata
3. **Search & Query** - Perform similarity searches
4. **Use FastAPI** - Access via REST API at http://localhost:8000/docs

### If Connection Failed ❌:
1. **Start Weaviate**: `docker-compose up -d`
2. **Wait 30 seconds** for initialization
3. **Check Docker**: `docker ps` (should see `weaviate_vectordb`)
4. **Check Ports**: Make sure 8080 and 50051 are not blocked
5. **Re-run this notebook** from the beginning

### Useful Commands:
```bash
# Start Weaviate
docker-compose up -d

# Check status
docker ps

# View logs
docker logs weaviate_vectordb

# Stop Weaviate
docker-compose down
```

### Documentation:
- **FastAPI App**: Run `python app_fastapi.py` then visit http://localhost:8000/docs
- **Weaviate Docs**: https://weaviate.io/developers/weaviate
- **Project README**: See `README_FASTAPI.md` for detailed setup guide